# Section 2.2 Data Structure Activities

In [ ]:
import os
import pymc3 as pm
import arviz as az
import pandas as pd
import numpy as np

if os.path.split(os.getcwd())[-1] != "notebooks":
    os.chdir(os.path.join(".."))
    
np.random.seed(0)

In [ ]:
az.style.use('arviz-white')

## Reproducing the Planet Experiment
More good news! Your astronomical discovery, from Section 1.2, has been published, but now people want to you to share your data and results. They also are asking for help getting seeing portions of your analysis runs to inspect in greater detail.

### Exercise 1 
Your favorite PPL is PyMC3, but it turns out your peer reviewer likes Stan. In an alternate universe your favorite PPL is Stan, but now your peer reviewer is a PyMC3 gal. Here we introduce the *Law of researcher PPL choice*  

$$P(\text{Your friends uses another PPL} | \text{You choice of PPL}) = 1$$


**How can we use ArviZ, Xarray, and NetCDF to share results in a common way?**  

Note: We encourage you to use whatever PPL you prefer. These docs may be helpful:

https://arviz-devs.github.io/arviz/api.html#data

#### Step 1: Define your model and generate results
The first step you'll need to take is to define your model and perform an inference run in this notebook. Recall in mathematical notation our model was as follows

$$\theta = \text{Proportion of water on the planet}$$

$$ 
\theta \sim \operatorname{Uniform}(0,1) \\
p_{\text{water}} \sim \operatorname{Binom}(\theta, N)
$$

In [ ]:
observations = [0, 0, 1, 0, 1]
water_observations = sum(observations)
total_observations = len(observations)

In [ ]:
# Add your model here, and perform an inference run. 
# You can either copy the model from Section 1.2 or for a challenge define your own

#### Step 2: Convert model results from PPL to az.InferenceData

In [ ]:
# Hint: Use the az.from_ family of methods

#### Step 3: Inspect InferenceData to see what groups exist

In [ ]:
# Inspect groups here

#### Step 4: Inspect Posterior group to verify variables count, chain count, and draw count

In [ ]:
# Inspect Posterior here

#### Step 3: Save your model to disk

In [ ]:
# Save your results here

### Exercise 2
You've been asked to peer review a study on radon levels in Minnesota basements. The dataset is available as part of ArviZ's remote datasets. You've been asked to do a couple things.

#### Step 1: Load the NetCDF file into python memory
*Note*: In ArviZ there are preloaded datasets. Radon is one of those. If you do not supply a dataset name to the correct method, an error message lists all the available ones.

In [ ]:
# Load Radon dataset. Hint look for the load_arviz_data

#### Step 2: List all the groups
See what analysis your colleague has already run by checking the groups present in the InferenceData object

In [ ]:
# Inspect groups here

#### Step 3: Count the number of counties included in radon study
How many counties were included in the observed_data?
Hint: xarray has a `.to_dataframe()` method

In [ ]:
# Use xarray and pandas index to determine number of counties

#### How many variables are in Bayesian model?
Inspect the posterior xarray dataset and get a list of data variables in xarray.

In [ ]:
# Use xarray to see how many data variables exist

#### Step 4: Select the first 10 values of chain 2 for sigma_y in the posterior
Using the `.sel` method get the first ten values 

In [ ]:
# Get First ten values